In [4]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [6]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [7]:
full_dataset = datasets.ImageFolder(r'C:\Radiologist_helper\datasets\ultrasound', transform=train_transform)

In [8]:
train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

In [10]:
val_dataset.dataset.transform = val_test_transform
test_dataset.dataset.transform = val_test_transform

In [18]:
print(train_dataset.dataset.class_to_idx)
print(test_dataset.dataset.class_to_idx)


{'benign': 0, 'malignant': 1, 'normal': 2}
{'benign': 0, 'malignant': 1, 'normal': 2}


In [11]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [12]:
class UltrasoundResNet50(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.3):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')

        # Freeze all layers
        for param in self.model.parameters():
            param.requires_grad = False

        # Optional: unfreeze last block (layer4)
        for param in self.model.layer4.parameters():
            param.requires_grad = True

        # Replace fully connected layer
        in_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [13]:
model = UltrasoundResNet50(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

In [13]:

def train_model(model, criterion, optimizer, epochs=10, patience=5, save_path=r"C:\Radiologist_helper\trained_models\Ultrasound_model.pth"):
    start = time.time()
    best_acc = 0.0
    best_loss = float('inf')
    patience_counter = 0

    all_labels, all_predictions = [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

            if (batch_num + 1) % 10 == 0:
                print(f"Batch: {batch_num+1}, Epoch: {epoch+1}, Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{epochs}] - Avg Train Loss: {epoch_loss:.4f}")

        # ---------- Validation ----------
        model.eval()
        correct, total = 0, 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        val_acc = 100 * correct / total
        print(f"Validation Loss: {val_loss:.4f} | Accuracy: {val_acc:.2f}%")

        # --- Early stopping and save ---
        if val_loss < best_loss:
            best_loss = val_loss
            best_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), save_path)
            print(f"Best model saved at epoch {epoch+1}")
        else:
            patience_counter += 1
            print(f"No improvement. Patience: {patience_counter}/{patience}")
            if patience_counter >= patience:
                break

    end = time.time()
    print(f"\nTraining complete in {(end - start):.2f}s")
    print(f"Best Validation Accuracy: {best_acc:.2f}%")

    return all_labels, all_predictions

In [14]:
all_labels, all_predictions = train_model(model, criterion, optimizer, epochs=10)

Batch: 10, Epoch: 1, Loss: 0.9313
Batch: 20, Epoch: 1, Loss: 0.5991
Batch: 30, Epoch: 1, Loss: 0.6570
Batch: 40, Epoch: 1, Loss: 0.8065
Batch: 50, Epoch: 1, Loss: 0.5615
Batch: 60, Epoch: 1, Loss: 0.6455
Batch: 70, Epoch: 1, Loss: 0.3932
Epoch [1/10] - Avg Train Loss: 0.6804
Validation Loss: 0.4389 | Accuracy: 80.89%
Best model saved at epoch 1
Batch: 10, Epoch: 2, Loss: 0.4333
Batch: 20, Epoch: 2, Loss: 0.3075
Batch: 30, Epoch: 2, Loss: 0.1899
Batch: 40, Epoch: 2, Loss: 0.4679
Batch: 50, Epoch: 2, Loss: 0.3582
Batch: 60, Epoch: 2, Loss: 0.3518
Batch: 70, Epoch: 2, Loss: 0.5394
Epoch [2/10] - Avg Train Loss: 0.3691
Validation Loss: 0.3417 | Accuracy: 89.17%
Best model saved at epoch 2
Batch: 10, Epoch: 3, Loss: 0.2570
Batch: 20, Epoch: 3, Loss: 0.3666
Batch: 30, Epoch: 3, Loss: 0.3468
Batch: 40, Epoch: 3, Loss: 0.2375
Batch: 50, Epoch: 3, Loss: 0.2838
Batch: 60, Epoch: 3, Loss: 0.3173
Batch: 70, Epoch: 3, Loss: 0.3254
Epoch [3/10] - Avg Train Loss: 0.2328
Validation Loss: 0.3074 | Accu

In [19]:
def evaluate_test(model, test_loader, model_path=r"C:\Radiologist_helper\trained_models\Ultrasound_model.pth"):
    device = next(model.parameters()).device
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (preds == labels).sum().item()

    test_acc = 100 * correct / total
    print(f"*** Test Accuracy: {test_acc:.2f}% ***")
    return test_acc


In [20]:
test_acc = evaluate_test(model, test_loader)
print(f"*** Test Accuracy: {test_acc:.2f}% ***")

*** Test Accuracy: 94.97% ***
*** Test Accuracy: 94.97% ***
